In [29]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [30]:
name = 'mnist-784-euclidean'
nameFull = 'mnist-784-euclidean-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (60000, 784)
testDataset :  (10000, 784)


<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [31]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = 8  # Number of subquantizers (bits per vector)
    nbits = 2*d  # Total number of bits for the PQ code
    time_start = perf_counter()
    index = indexMethod(d, M, nbits, faiss.METRIC_L2)
    index.train(datasetImages)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexPQ, datasetTrainImages)

Took 0.930 seconds


In [32]:
print(indexedStruct)

<faiss.swigfaiss.IndexPQ; proxy of <Swig Object of type 'faiss::IndexPQ *' at 0x0000018A0E6F4D50> >


In [33]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [34]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

KeyboardInterrupt: 

In [ ]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n', 'max : ', max,)

In [ ]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [ ]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [ ]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [ ]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

0 || 53843
1674.2998 || 676.584
1 || 38620
1674.2998 || 793.9868
2 || 16186
1674.2998 || 862.6766
3 || 27059
1674.2998 || 864.5039
4 || 47003
1674.2998 || 894.7
5 || 14563
1674.2998 || 909.7043
6 || 44566
1674.2998 || 917.6323
7 || 15260
1674.2998 || 921.6241
8 || 40368
1674.2998 || 922.147
9 || 36395
1674.2998 || 943.4972


In [ ]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.0499
0.0625
0.2401
